# Project Proposal

### Requirements:

* 1 page (max 500 words) written proposal 
* proposal should be done in a Jupyter notebook, and then submitted both as an .html file (File -> Download As -> HTML) and an .ipynb file that is reproducible 

### Each proposal should include the following sections:

* Title
* Introduction:
(a) Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal. 
(b) Clearly state the question you will try to answer with your project. 
(c) Identify and describe the dataset that will be used to answer the question.
* Preliminary exploratory data analysis:
(a) Demonstrate that the dataset can be read from the web into R. 
(b) Clean and wrangle your data into a tidy format.
(c) Using only training data, summarize the data in at least one table (this is exploratory data analysis). (ci) An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
(d) Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). (di) An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.
* Methods:
(a) Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
(b) Describe at least one way that you will visualize the results
* Expected outcomes and significance:
(a) What do you expect to find?
(b) What impact could such findings have?
(c) What future questions could this lead to?

Take note of grading criteria at end: https://canvas.ubc.ca/courses/106510/assignments/1419852?module_item_id=5168273